In [1]:
import numpy as np
import matplotlib.pyplot as plt
import paddle
paddle.set_default_dtype("float64")

In [2]:
class RandomDataset(paddle.io.Dataset):
    def __init__(self,L,b,n,s):
        self.train_data=np.random.rand(n,len(L))
        self.M=np.array(L).T
        self.train_label=(self.train_data@self.M+b+np.random.rand(n)*s).reshape(-1,1)
    def __getitem__(self,idx):
        return self.train_data[idx],self.train_label[idx]
    def __len__(self):
        return len(self.train_label)

In [3]:
L=[0,1,2,3,4,5,6,7,8,9]
b=10
n=10000
s=1
batch_size=10
iterations=100

rd=RandomDataset(L,b,n,s)
train_loader=paddle.io.DataLoader(rd,batch_size=batch_size)

In [4]:
class Regressor(paddle.nn.Layer):
    def __init__(self,n):
        super(Regressor,self).__init__()
        self.li=paddle.nn.Linear(n,1,None)
    def forward(self,inputs):
        return self.li(inputs)

model=Regressor(len(L))
mse_loss=paddle.nn.MSELoss()
opt=paddle.optimizer.SGD(learning_rate=0.0005, parameters=model.parameters())
#opt=paddle.optimizer.Adam(learning_rate=0.0005, parameters=model.parameters())
#opt=paddle.optimizer.Lamb(learning_rate=0.0005, parameters=model.parameters())
#opt=paddle.optimizer.Momentum(learning_rate=0.0005, parameters=model.parameters())

In [5]:
model.train()
for i in range(iterations):
    loss=0
    for data,label in train_loader:
        pred=model(data)
        loss=mse_loss(pred,label)
        loss.backward()
        opt.step()
        opt.clear_grad()
    print(i,loss.numpy())

0 9.508655643505131
1 5.481207177039447
2 4.692515694245772
3 4.068389815926629
4 3.5345415453071345
5 3.0764649762445466
6 2.6830773563787647
7 2.344971004304464
8 2.0541302894075315
9 1.8037228805283327
10 1.5879231925559285
11 1.4017623607299825
12 1.2410007231530769
13 1.1020194202663334
14 0.9817282327839537
15 0.877487214211729
16 0.7870400429474527
17 0.708457332050566
18 0.6400884005239382
19 0.580520235534424
20 0.5285425664956837
21 0.48311813450229113
22 0.443357378618244
23 0.408496877698474
24 0.37788098591173197
25 0.3509461846096087
26 0.32720774492316035
27 0.3062483563862828
28 0.2877084286221871
29 0.27127781707183535
30 0.2566887610673471
31 0.24370985425939323
32 0.23214089434384663
33 0.22180848191749453
34 0.21256225773858373
35 0.2042716841922812
36 0.19682329080525746
37 0.19011831559018488
38 0.1840706841489075
39 0.17860527708968332
40 0.17365644364977248
41 0.1691667256534261
42 0.16508576124116225
43 0.16136934232057581
44 0.15797860352923285
45 0.1548793237

In [6]:
model.parameters()

[Parameter containing:
 Tensor(shape=[10, 1], dtype=float64, place=Place(cpu), stop_gradient=False,
        [[0.02335404],
         [1.02622977],
         [2.02421172],
         [3.01754129],
         [4.01619847],
         [5.01962098],
         [6.03175358],
         [7.02045699],
         [8.03595942],
         [9.02049585]]),
 Parameter containing:
 Tensor(shape=[1], dtype=float64, place=Place(cpu), stop_gradient=False,
        [10.37411993])]

In [7]:
paddle.save(model.state_dict(),"a")

In [8]:
model1=Regressor(len(L))
model1.set_state_dict(paddle.load("a"))
model1.eval()
eval_loader=paddle.io.DataLoader(RandomDataset(L,b,n,s),batch_size=batch_size)

In [9]:
Loss=[]
for data,label in eval_loader:
    pred=model1(data)
    Loss.append(mse_loss(pred,label).numpy())
    #print(pred.numpy()[0],label.numpy()[0])
    #print(Loss[-1])
print(f"\nMSE = {np.mean(Loss)}")


MSE = 0.08386316538597068
